<div dir=rtl>


در این تمرین، یک مدل درخت تصمیم به‌صورت دستی برای داده‌های معروف Iris پیاده‌سازی شد. ابتدا داده‌ها نرمال‌سازی شدند و سپس با استفاده از تابعی بازگشتی، درخت تصمیم با معیار Gini ساخته شد. برای هر گره، بهترین ویژگی و مقدار آستانه به‌صورت بهینه انتخاب شد تا داده‌ها به صورت حداکثری از هم جدا شوند.

بعد از ساخت مدل، آن را روی مجموعه تست ارزیابی کردیم و معیارهای مختلفی از جمله دقت، precision، recall و f1-score محاسبه شدند. تمام این مقادیر بالا بودند که نشان می‌دهد درخت تصمیم توانسته به‌خوبی ساختار داده را یاد بگیرد و تعمیم بدهد.

ساختار نهایی درخت نیز ساده ولی مؤثر بود؛ به‌گونه‌ای که کلاس Setosa تنها با یک شرط از بقیه جدا شد و برای تفکیک Versicolor و Virginica از دو ویژگی دیگر با چند تقسیم اضافی استفاده شد. این نتایج نشان می‌دهد که درخت تصمیم با عمق محدود هم توانسته ساختار داده‌ها را به‌خوبی یاد بگیرد و دسته‌بندی دقیقی انجام دهد.


</div>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
# from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler

In [ ]:



iris = load_iris()
X, y = iris.data, iris.target
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, stratify=y, random_state=42
)

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

def gini(y):
    m = len(y)
    if m == 0:
        return 0
    _, counts = np.unique(y, return_counts=True)
    probs = counts / m
    return 1 - np.sum(probs ** 2)

def best_split(X, y):
    best_feat, best_thresh, best_imp = None, None, float('inf')
    n_features = X.shape[1]
    for f in range(n_features):
        for t in np.unique(X[:, f]):
            left = y[X[:, f] < t]
            right = y[X[:, f] >= t]
            if len(left) == 0 or len(right) == 0:
                continue
            imp = (len(left) * gini(left) + len(right) * gini(right)) / len(y)
            if imp < best_imp:
                best_imp, best_feat, best_thresh = imp, f, t
    return best_feat, best_thresh

def build_tree(X, y, max_depth=5, min_size=5, depth=0):
    if len(set(y)) == 1 or len(y) < min_size or depth >= max_depth:
        labels, counts = np.unique(y, return_counts=True)
        return Node(value=labels[np.argmax(counts)])
    feat, thresh = best_split(X, y)
    if feat is None:
        labels, counts = np.unique(y, return_counts=True)
        return Node(value=labels[np.argmax(counts)])
    mask = X[:, feat] < thresh
    left = build_tree(X[mask], y[mask], max_depth, min_size, depth+1)
    right = build_tree(X[~mask], y[~mask], max_depth, min_size, depth+1)
    return Node(feature=feat, threshold=thresh, left=left, right=right)

def predict_one(x, node):
    if node.value is not None:
        return node.value
    if x[node.feature] < node.threshold:
        return predict_one(x, node.left)
    else:
        return predict_one(x, node.right)

def predict(X, tree):
    return np.array([predict_one(x, tree) for x in X])

def print_tree(node, indent=""):
    if node.value is not None:
        print(indent + "Predict ->", node.value)
        return
    print(indent + f"[X[{node.feature}] < {node.threshold:.3f}]")
    print(indent + "-->True:")
    print_tree(node.left, indent + "    ")
    print(indent + "-->False:")
    print_tree(node.right, indent + "    ")

tree = build_tree(X_train, y_train)
y_pred = predict(X_test, tree)

print("accuracy      :", accuracy_score(y_test, y_pred))
print("precision (µ) :", precision_score(y_test, y_pred, average='macro'))
print("recall   (µ)  :", recall_score(y_test, y_pred, average='macro'))
print("f1-score (µ)  :", f1_score(y_test, y_pred, average='macro'))
print()
print_tree(tree)


accuracy      : 0.9666666666666667
precision (µ) : 0.9696969696969697
recall   (µ)  : 0.9666666666666667
f1-score (µ)  : 0.9665831244778612

[X[2] < -0.431]
-->True:
    Predict -> 0
-->False:
    [X[3] < 0.659]
    -->True:
        [X[2] < 0.706]
        -->True:
            Predict -> 1
        -->False:
            Predict -> 2
    -->False:
        [X[2] < 0.649]
        -->True:
            Predict -> 2
        -->False:
            Predict -> 2
